# Symbolic Regression mono-objective

#### 1. Libraries importation

In [1]:
import math
import random
import csv
import numpy
import operator
import matplotlib.pyplot as plt
from deap import algorithms, base, creator , tools, gp

#### 2. Defining primitive set

In [2]:
# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

#Second argument = number of variables in problem (this case, 'x')
pset = gp.PrimitiveSet("MAIN", 1)
#Second argument = arity
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-10,10))
pset.renameArguments(ARG0='x')

#### 3. Parameters definition

In [3]:
#Defining fitness class
creator.create("FitnessMin", base.Fitness, weights=(-1,))
#Defining individuals shape and associatinf fitness attribute
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

#Creating toolbox to register: population creation, evaluation function, selection mecanism
#and genetic operators
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    #sqerrors = ((func(x) - x**4 - x**3 - x**2 - x )**2 for x in points)
    sqerrors = ((func(x) - (math.sin(x**2))**2 - math.sin(x) - (math.cos(x**2))**2 - math.cos(x) - x**3 - 2*x**2 - 4 )**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x for x in range(-100,100)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

#### 4. Algorithm initialization

In [4]:
def main():
    random.seed(318)

    pop = toolbox.population(n=500)
    hof = tools.HallOfFame(1)

    stats_fit_mse = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(mse=stats_fit_mse, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log, hof = algorithms.eaSimpleOr(pop, toolbox, .8, .1, 500, stats=mstats,
                                   halloffame=hof, verbose=True)
    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()

Creating pandas dataframe


,mse_avg,mse_std,mse_min,mse_max,mse_gen,mse_nevals,size_avg,size_std,size_min,size_max,size_gen,size_nevals
0,1.427664e+11,1.215847e+08,1.415256e+11,1.442270e+11,1.0,457.0,3.922,1.649217,1.0,10.0,1.0,457.0
1,1.427619e+11,1.879413e+08,1.421482e+11,1.445879e+11,2.0,456.0,4.218,1.941771,1.0,15.0,2.0,456.0
2,1.427512e+11,2.224081e+08,1.421482e+11,1.456025e+11,3.0,446.0,4.438,1.869266,1.0,11.0,3.0,446.0
3,1.427164e+11,2.714701e+08,1.408358e+11,1.453195e+11,4.0,444.0,4.770,2.095018,1.0,13.0,4.0,444.0
4,1.426655e+11,3.909505e+08,1.402812e+11,1.472302e+11,5.0,440.0,5.476,2.521393,1.0,17.0,5.0,440.0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,4.180249e+13,9.336432e+14,1.810175e-01,2.089781e+16,496.0,455.0,150.378,29.459245,4.0,240.0,496.0,455.0
496,1.136301e+15,2.538289e+16,1.810175e-01,5.681472e+17,497.0,445.0,150.502,27.231636,1.0,212.0,497.0,445.0
497,2.229276e+12,4.964464e+13,1.810175e-01,1.111206e+15,498.0,459.0,153.122,25.001022,9.0,212.0,498.0,459.0
498,2.793773e+14,6.240685e+15,1.810175e-01,1.396857e+17,499.0,446.0,150.814,28.380546,4.0,217.0,499.0,446.0


### 5. Simplifying resultant equation

In [5]:
from sympy import sympify, sin, cos
def protectedDiv(left, right):
    try:
        return left / right
    except:
        return 1
locals = {
    'sub': lambda x, y : x - y,
    'protectedDiv': protectedDiv,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y, 
    'sin': lambda x: sin(x), 
    'cos': lambda x: cos(x)
}
ind = hof.__getitem__(0).__str__()
print(f'original: {ind}')
expr = sympify(ind , locals=locals)
print(f'simplified: {expr}')


original: mul(add(protectedDiv(sub(protectedDiv(add(x, mul(sin(0), x)), protectedDiv(sin(0), protectedDiv(-1, protectedDiv(mul(8, -10), protectedDiv(protectedDiv(sin(add(neg(x), x)), x), neg(x)))))), 6), add(neg(x), protectedDiv(add(neg(x), protectedDiv(add(add(neg(x), protectedDiv(add(neg(add(x, add(neg(0), cos(add(x, -6))))), protectedDiv(neg(x), protectedDiv(-1, sin(x)))), 3)), protectedDiv(neg(x), protectedDiv(-1, sin(sin(sin(add(add(x, sin(sin(-4))), sin(mul(cos(x), cos(x)))))))))), protectedDiv(-4, sin(add(x, sin(-4)))))), protectedDiv(-4, sin(add(x, sin(protectedDiv(mul(sin(2), protectedDiv(x, 1)), x)))))))), add(x, add(x, mul(x, x)))), x)
simplified: nan


In [12]:
from sympy import sympify, sin, cos

locals = {
    'sub': lambda x, y : x - y,
    'protectedDiv': protectedDiv,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y, 
    'sin': lambda x: sin(x), 
    'cos': lambda x: cos(x)
}
expr = sympify(ind, locals = locals)
print(expr)

TypeError: unsupported operand type(s) for /: 'Singleton' and 'Mul'

### 6. Graphs

In [ ]:
fig, ax = plt.subplots()
original = lambda x: (math.sin(x**2))**2 + math.sin(x) + (math.cos(x**2))**2 + math.cos(x) + x**3 + 2*x**2 + 4
ax.plot(numpy.linspace(-100,100,100), [original(x) for x in numpy.linspace(-100,100,100)], color = 'r', marker = 'o', label='original')
aprox = lambda x: toolbox.compile(hof.__getitem__(0))(x)
ax.plot(numpy.linspace(-100,100,100), [aprox(x) for x in numpy.linspace(-100,100,100)], color = 'b', marker='v', label='aproximation')
plt.legend()